In [ ]:
from pyspark.sql import SparkSession, DataFrame as SparkDataFrame
import os

# Create a SparkSession
spark = SparkSession.builder.appName("DataFrame-Operations").getOrCreate()
import datetime
start = datetime.datetime.now()
try:
    __file__ = __file__ 
except:
    __file__ = os.path.join(os.getcwd(), 'DataFrame_Operations.ipynb')

In [ ]:
# Load the synthetic data into a DataFrame
import os
directory = os.path.dirname(__file__)
data_file_path = os.path.join( directory, "../" , "data/stocks.txt")

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

schema = StructType(
  [
    StructField("id", IntegerType()),
    StructField("name", StringType()),
    StructField("category", StringType()),
    StructField("quantity", IntegerType()),
    StructField("price", DoubleType()),
  ]
)

df = spark.read.schema(schema).csv(data_file_path, header=True, inferSchema=True)

In [ ]:
# Display schema of DataFrame
df.printSchema()

# Show the initial DataFrame
print("Initial DataFrame:")
df.show(10)

### Select: Choose specific columns.

In [ ]:
# Select specific columns
selected_columns = df.select("id", "name", "price")
print("Selected Columns:")
selected_columns.show(10)

### Filter: Apply conditions to filter rows.

In [ ]:
from pyspark.sql.functions import avg, round as round_, col, max as max_

def filter_products():
  items_df = df.filter(df.name.rlike("Headphones$") | (df.name == "iPhone"))
  total_items  = round(items_df.count() * 0.10)
  combo_discount_df = spark.createDataFrame(items_df.orderBy(col("price").asc()).take(total_items)).groupBy().agg(max_("price").alias("two_items_discount"))
  inventory = (items_df.groupBy("name").agg(round_(avg("price") * 0.15).alias("discount")).select([col('name').alias('inv_name'), col('discount')])
               .join(combo_discount_df))
  inventory.show()
  discounts_df = items_df.join(inventory, on=(items_df.name == inventory.inv_name)).drop('inv_name') 
  return discounts_df
  

filtered_data = filter_products()

from snowflake.snowpark_checkpoints_collector import collect_dataframe_checkpoint
collect_dataframe_checkpoint(filtered_data, checkpoint_name="filtered_data", sample=0.1, file_dir=os.path.dirname(__file__))

total_price = filtered_data.select(avg(filtered_data.price))

filtered_data.show()
filtered_data.select("name").distinct().show()
total_price.show()

In [ ]:
def first_sport_item(products_df):  
  products_df = products_df.repartition(5, df.name)
  result = products_df.where(products_df.category == 'Sports')
  result = result.orderBy("price")
  return spark.createDataFrame([result.first()]) 

first_sport_df = first_sport_item(df)

first_sport_df.show()

from snowflake.snowpark_checkpoints_collector import collect_dataframe_checkpoint
collect_dataframe_checkpoint(first_sport_df, checkpoint_name="first_sport_item", sample=1.0, file_dir=os.path.dirname(__file__))



In [ ]:
# Filter rows based on a condition
filtered_data = df.filter(df.quantity > 20)
print("Filtered Data:", filtered_data.count())
filtered_data.show()

### GroupBy: Group data based on specific columns 
### Aggregations: Perform functions like sum, average, etc., on grouped data.

In [ ]:
# GroupBy and Aggregations
grouped_data = df.groupBy("category").agg({"quantity": "sum", "price": "avg"})
print("Grouped and Aggregated Data:")
grouped_data.show()

### Join: Combine multiple DataFrames based on specified columns.

In [ ]:
# Join with another DataFrame
df2 = df.select("id", "category").limit(10)
joined_data = df.join(df2, "id", "inner")
print("Joined Data:")
joined_data.show()

### Sort: Arrange rows based on one or more columns.

In [ ]:
# Sort by a column
sorted_data = df.orderBy("price")
print("Sorted Data:")
sorted_data.show(10)

In [ ]:
# Sort by a column desc
from pyspark.sql.functions import col, desc
sorted_data = df.orderBy(col("price").desc(), col("id").desc())
print("Sorted Data Descending:")
sorted_data.show(10)

### Distinct: Get unique rows.

In [ ]:
# Get distinct product category
distinct_rows = df.select("category").distinct()
print("Distinct Product Categories:")
distinct_rows.show()

### Drop: Remove specified columns.

In [ ]:
# Drop columns
dropped_columns = df.drop("quantity", "category")
print("Dropped Columns:")
dropped_columns.show(10)

### WithColumn: Add new calculated columns.

In [ ]:
# Add a new calculated column
df_with_new_column = df.withColumn("revenue", df.quantity * df.price)
print("DataFrame with New Column:")
df_with_new_column.show(10)

### Alias: Rename columns for better readability.

In [ ]:
# Rename columns using alias
df_with_alias = df.withColumnRenamed("price", "product_price")
print("DataFrame with Aliased Column:")
df_with_alias.show(10)

In [ ]:
end = datetime.datetime.now()

print((end - start))